<a href="https://colab.research.google.com/github/akki0894/Algo-Trading/blob/main/S%26P_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [104]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [105]:
!pip install xlsxwriter

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [106]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [107]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [108]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 73934664, 'calculationPrice': 'close', 'change': -0.85, 'changePercent': -0.00581, 'close': 149.32, 'closeSource': 'laiicfof', 'closeTime': 1705989439949, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 155.6, 'delayedPriceTime': 1703483740746, 'extendedChange': -0.53, 'extendedChangePercent': -0.00357, 'extendedPrice': 151.14, 'extendedPriceTime': 1693289226573, 'high': 157, 'highSource': 'pddmlairt1 y ue5ne ecei', 'highTime': 1679312102594, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 156.23, 'iexCloseTime': 1694908201543, 'iexLastUpdated': 1644605863007, 'iexMarketPercent': 0.011001737727080034, 'iexOpen': 153.27, 'iexOpenTime': 1658383868520, 'iexRealtimePrice': 151.85, 'iexRealtimeSize': 30, 'iexVolume': 804157, 'lastTradeTime': 1681637832788, 'latestPrice': 149.87, 'latestSource': 'Close', 'latestTime': 'November 1, 2021', 'latestUpdate': 1692429197360, 'latestVolume': 78236930, 'low': 149, 'lowSource': ' eyec

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [109]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/100000000)

25810.05313559


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [110]:
my_columns = ['Ticker','Stock_Price','Market Capitalization','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock_Price,Market Capitalization,Number of Shares to Buy


In [111]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock_Price,Market Capitalization,Number of Shares to Buy
0,AAPL,149.87,2581005313559,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [112]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
  api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(api_url).json()
  final_dataframe=final_dataframe.append(
      pd.Series(
          [
           stock,
           data['latestPrice'],
           data['marketCap'],
           'N/A'
          ],
          index = my_columns),
        ignore_index =True
      )

In [113]:
final_dataframe

,Ticker,Stock_Price,Market Capitalization,Number of Shares to Buy
0,A,157.10,48135781613,N/A
1,AAL,20.64,12939608291,N/A
2,AAP,232.36,14367030688,N/A
3,AAPL,152.01,2530733453076,N/A
4,ABBV,118.22,211808553765,N/A
...,...,...,...,...
500,YUM,130.23,38165602763,N/A
501,ZBH,151.41,31366329403,N/A
502,ZBRA,547.91,29116947908,N/A
503,ZION,66.23,10540681033,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [114]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [115]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock_Price,Market Capitalization,Number of Shares to Buy
0,A,159.54,47829531961,N/A
1,AAL,20.17,13317813300,N/A
2,AAP,235.01,14257215980,N/A
3,AAPL,150.34,2488036921808,N/A
4,ABBV,119.31,211919494790,N/A
...,...,...,...,...
500,YUM,130.40,37649435446,N/A
501,ZBH,147.46,31570238611,N/A
502,ZBRA,569.37,29337571950,N/A
503,ZION,64.24,10598961453,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [116]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:10000000


In [117]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock_Price'])
final_dataframe

,Ticker,Stock_Price,Market Capitalization,Number of Shares to Buy,Number Of Shares to Buy
0,A,159.54,47829531961,N/A,124.0
1,AAL,20.17,13317813300,N/A,981.0
2,AAP,235.01,14257215980,N/A,84.0
3,AAPL,150.34,2488036921808,N/A,131.0
4,ABBV,119.31,211919494790,N/A,165.0
...,...,...,...,...,...
500,YUM,130.40,37649435446,N/A,151.0
501,ZBH,147.46,31570238611,N/A,134.0
502,ZBRA,569.37,29337571950,N/A,34.0
503,ZION,64.24,10598961453,N/A,308.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [118]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [119]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [120]:
writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

0

In [121]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [122]:
writer.save()